# Ant Problem

This notebook tries to provide a solution to the following question:

An ant leaves its anthill in order to forage for food. It moves with the speed of 10cm per second, but it doesn’t know where to go, therefore every second it moves randomly 10cm directly north, south, east or west with equal probability.

- If the food is located on east-west lines 20cm to the north and 20cm to the south, as well as on north-south lines 20cm to the east and 20cm to the west from the anthill, how long will it take the ant to reach it on average?
- What is the average time the ant will reach food if it is located only on a diagonal line passing through (10cm, 0cm) and (0cm, 10cm) points?
- Can you Write a program that comes up with an estimate of average time to find food for any closed boundary around the anthill? What would be the answer if food is located outside an area defined by ( (x – 2.5cm) / 30cm )2 + ( (y – 2.5cm) / 40cm )2 < 1 in coordinate system where the anthill is located at (x = 0cm, y = 0cm)?


## Base Code

For simulating different parts of the problem, I created a simple random walk function that gets a goal condition and tries to reach that goal randomly. At the end it returns the steps it used for reaching the goal. The `simulate` function runs the random walk multiple times and returns the average steps needed to rach the goal 

In [1]:
import math
import random
from typing import Callable
from tqdm import tqdm

In [2]:
def random_walk_step(pos: tuple[int, int]) -> tuple[int, int]:
    movement = random.choice([
        (10, 0),
        (-10, 0),
        (0, 10),
        (0, -10),
    ])
    return (pos[0] + movement[0], pos[1] + movement[1])

In [3]:
def random_walk(
        start_pos: tuple[int, int],
        condition: Callable[[tuple[int, int]], bool],
        max_steps: int = 10000,
) -> int | float:
    steps = 0
    pos = start_pos
    while not condition(pos) and steps < max_steps:
        pos = random_walk_step(pos=pos)
        steps += 1

    if steps == max_steps:
        return math.inf

    return steps

In [4]:
def simulate(
        start_pos: tuple[int, int],
        condition: Callable[[tuple[int, int]], bool],
        simulation_rounds: int = 10000,
) -> float:
    results = []
    for _ in tqdm(range(simulation_rounds)):
        results.append(
            random_walk(
                start_pos=start_pos,
                condition=condition,
            )
        )

    return sum(results) / len(results)

## Question 1

The problem is very similar to the famous gambler's ruin. We can use the Markov chain theorem to solve it.


$E(T_{i,j}^{x=+20}) = Expected\ time\ to\ reach\ x = +20cm\ if\ we\ are\ currently\ at\ x=i\ and\ y=j$

So the goal is to find $E{T_{0,0}^{x=+20}}$ here. Based on the Markov theorem we can assume the following:

$E(T_{i,j}^{x=+20}) = 1 + \frac{1}{4}E(T_{i-10,j}^{x=+20}) + \frac{1}{4}E(T_{i+10,j}^{x=+20}) + \frac{1}{4}E(T_{i,j-10}^{x=+20}) + \frac{1}{4}E(T_{i,j+10}^{x=+20})$

We also know:

$\forall i, j \in R, E(T_{i,20}^{x=+20}) = E(T_{20,j}^{x=+20}) = E(T_{-20, j}^{x=+20}) = E(T_{i,-20}^{x=+20}) = 0$

By solving the above equation system, and also the same systems for $E{T_{0,0}^{x=-20}}, E{T_{0,0}^{y=+20}}\ and\ E{T_{0,0}^{y=-20}}$, we can compute the final estimated time with the sum.

$E(T_{0,0}) = E{T_{0,0}^{x=+20}} + E{T_{0,0}^{x=-20}} + E{T_{0,0}^{y=+20}} + E{T_{0,0}^{y=-20}} = 4.5$

In [5]:
def question_1_condition(pos: tuple[int, int]) -> bool:
    return abs(pos[0]) >= 20 or abs(pos[1]) >= 20

In [6]:
q1_solution = simulate(
    start_pos=(0, 0),
    condition=question_1_condition,
)
print(f'Average time to reach the goal is: {q1_solution}.')

100%|██████████| 10000/10000 [00:00<00:00, 363202.95it/s]

Average time to reach the goal is: 4.4932.


## Question 2

The answer is infinity, Because the expected time to reach an unbounded goal in random walk is **$\infty$**.

In [7]:
def question_2_condition(pos: tuple[int, int]) -> bool:
    return pos[0] + pos[1] == 10

In [8]:
q2_solution = simulate(
    start_pos=(0, 0),
    condition=question_2_condition,
)
print(f'Average time to reach the goal is: {q2_solution}.')

100%|██████████| 10000/10000 [00:00<00:00, 11945.65it/s]

Average time to reach the goal is: inf.


## Question 3

We can define the goal condition function like bellow and we get the value of **~14.0** as the estimated steps needed for the ant to reach the food.

In [9]:
def question_3_condition(pos: tuple[int, int]) -> bool:
    x, y = pos
    return math.pow((x - 2.5) / 30, 2) + math.pow((y - 2.5) / 40, 2) >= 1

In [10]:
q3_solution = simulate(
    start_pos=(0, 0),
    condition=question_3_condition,
)
print(f'Average time to reach the goal is: {q3_solution}.')

100%|██████████| 10000/10000 [00:00<00:00, 79954.71it/s]

Average time to reach the goal is: 13.9417.
